# Instalación de dependencias para librerías y bases de datos

In [1]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!git clone https://github.com/UN-GCPDS/eeg_models.git
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.

  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-s7mf3nvs
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-s7mf3nvs
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit d174df9958b6638156dcfe03996a6307e631a6a2
  Preparing metadata (setup.py) ... done
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=32972807 sha256=ebacc0160c4dcb46df6895cf19a5c3c70613e3d1234f8939abfbfed190a74e88
  Stored in directory: /tmp/pip-ephem-wheel-cache-08kzzl6p/wheels/ae/48/8d/edf617d5fe8f03b17aa26306a04abdfcc605b218d8e6deac83
Successfully built gcpds-databases
Cloning into 'eeg_models'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 134 (delta 81), reused 72 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (134/134), 79.89 KiB | 7.

# Importación de librerías

In [2]:
# load databases
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import sys
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
sys.path.append('/kaggle/working/eeg_models')

from utils import train
import numpy as np

2025-04-25 13:11:24.184230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745586684.386458      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745586684.446742      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Cración del modelo

In [3]:
from models import TCNet_fusion

# Definición de las dimensiones de entrada
C = 64  # Número de canales
T = 320  # Número de muestras por segmento

# Parámetros específicos del modelo
model_params = {'nb_classes' : 2,
                'layers' : 2,
                'kernel_s' : 4,
                'filt' : 12,
                'dropout' : 0.3,
                'activation' : 'relu',
                'F1' : 24,
                'D' : 2,
                'kernLength' : 32,
                'N_residuals' : 2}

# Argumentos generales del modelo
model_args = {
    'model_name': 'TCNet_fusion',
    'nb_classes': 2,
    'autoencoder': False
}

# Inicialización del modelo con los parámetros definidos
model = TCNet_fusion(**model_params, Chans=C, Samples=T)

model.summary()

I0000 00:00:1745586695.777206      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745586695.777910      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)        │ (None, 64, 320, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 320, 64, 1)     │              0 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2D_1 (Conv2D)         │ (None, 320, 64, 24)    │            768 │ permute[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 320, 64, 24)    │             96 │ Conv2D_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Depth_wise_Conv2D_1       │ (None, 320, 1, 48)     │          3,072 │ batch_normalization[0… │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 320, 1, 48)     │            192 │ Depth_wise_Conv2D_1[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 320, 1, 48)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d         │ (None, 40, 1, 48)      │              0 │ activation[0][0]       │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 40, 1, 48)      │              0 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Separable_Conv2D_1        │ (None, 40, 1, 48)      │          3,072 │ dropout[0][0]          │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 40, 1, 48)      │            192 │ Separable_Conv2D_1[0]… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 40, 1, 48)      │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d_1       │ (None, 5, 1, 48)       │              0 │ activation_1[0][0]     │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 5, 1, 48)       │              0 │ average_pooling2d_1[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 5, 48)          │              0 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 5, 12)          │          2,316 │ lambda[0][0]           │
├──────────────────────

 Total params: 16,034 (62.63 KB)

 Trainable params: 15,602 (60.95 KB)

 Non-trainable params: 432 (1.69 KB)

# Pruebas con GIGA

In [4]:
# Nombre de la base de datos y carga
db_name = 'GIGA_MI_ME'
db_path = '/kaggle/input/giga-science-gcpds/GIGA_MI_ME'
db = GIGA_MI_ME(db_path)

# Frecuencia de muestreo
fs = db.metadata['sampling_rate']

# Nombres de los canales EEG
eeg_ch_names = [
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

# Definición de los bancos de frecuencia y ventana temporal
f_bank = np.array([[4., 40.]])        # Banda de frecuencia de interés
vwt = np.array([[2.5, 5]])            # Ventana temporal para la representación TF

# Representación tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# Argumentos para la función de carga de datos
load_args = {
    'db': db,
    'eeg_ch_names': eeg_ch_names,
    'fs': fs,
    'f_bank': f_bank,
    'vwt': vwt,
    'new_fs': 128.0,
    'tf_repr': tf_repr
}

# Selección de sujetos (excluyendo sujetos 29 y 34)
subjects = np.arange(1, db.metadata['subjects'] + 1)
subjects = np.delete(subjects, [28, 33])

In [5]:
# === Imports ===
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

# === Parámetros generales ===
seed = 23
verbose = 0

# === Callbacks de entrenamiento ===
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=30,
    verbose=verbose,
    mode='min',
    min_delta=0.01,
    min_lr=0
)

terminate_on_nan = TerminateOnNaN()

callbacks = [reduce_lr_on_plateau, terminate_on_nan]

# === Configuración de validación cruzada ===
cv_args = {
    'cv': StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
}

# === Argumentos de compilación del modelo ===
compile_args = {
    'loss': SparseCategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 1e-2
}

# === Argumentos del ajuste (fit) del modelo ===
fit_args = {
    'epochs': 500,
    'verbose': verbose,
    'callbacks': callbacks
}

In [6]:
all_results = []

for sbj in subjects[:15]: 
    print('sbj = ', sbj)
    load_args['sbj'] = sbj
    results = train(model, db_name, tf_repr, load_args, cv_args, model_args, compile_args, fit_args, seed)
    all_results.append((sbj, results))

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.


I0000 00:00:1745586715.774332     120 service.cc:148] XLA service 0x7f4254002d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745586715.775110     120 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745586715.775128     120 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745586716.860145     120 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-25 13:12:01.911977: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng2{k2=3,k3=0} for conv (f32[160,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[160,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_s

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  2
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  3
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
sbj =  4
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  5
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 13:35:37.692163: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng2{k2=3,k3=0} for conv (f32[159,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 13:35:38.063402: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.371361129s
Trying algorithm eng2{k2=3,k3=0} for conv (f32[159,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convFor

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  6
Resampling from 512.000000 to 128.000000 Hz.


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  9
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  10
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  11
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  12
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 14:18:23.854125: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng2{k2=3,k3=0} for conv (f32[139,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[139,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 14:18:24.061924: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.207865846s
Trying algorithm eng2{k2=3,k3=0} for conv (f32[139,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[139,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convFor

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  13
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  14
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
sbj =  15
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 14:36:08.608846: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng2{k2=3,k3=0} for conv (f32[152,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[152,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 14:36:08.914032: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.305355451s
Trying algorithm eng2{k2=3,k3=0} for conv (f32[152,24,320,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[152,48,320,1]{3,2,1,0}, f32[24,2,1,64]{3,2,1,0}), window={size=1x64 pad=0_0x63_63}, dim_labels=bf01_oi01->bf01, feature_group_count=24, custom_call_target="__cudnn$convFor

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [7]:
import pickle

with open(f'results_{model_args["model_name"]}_1.pkl', 'wb') as f:
    pickle.dump(all_results, f)